<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq

In [163]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [164]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [165]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [166]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [167]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [168]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [169]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-02 11:39:33
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.34 C
-------------------
Today PnL: -2.08 L (-1.52%)
Current PnL: -35.05 L (-22.35%)
CY Booked + Current PnL: -20.31 L (-12.95%)
-------------------
Total profit:  79.13 K
Total loss:  -35.84 L
-------------------
Total Booked + Current PnL: 6.67 L (5.14%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.96%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 1.01 C (74.79%)
Deployed:  1.30 C
Current:  1.34 C
CAGR/XIRR %: 1.82%


In [170]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.62,2.30,16.55,19.23,25474.0,3460.0,153920.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.15,24.06,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.09,-26.44,36.07,0.09,26740.0,-26644.0,74133.0,72.04,33.0,M-SC,6.50,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
50,MASFIN,398.61,-6.53,-8.14,32.87,22.05,29583.0,-7980.0,90000.0,-20.96,42.0,H-SC,3.38,168.0,-0.27,0.67,30.82,XR,ATH,FINANCE
33,HCLTECH,1908.19,-2.27,9.19,13.41,23.83,35423.0,22228.0,264152.0,14.03,53.0,X-LC,7.32,13.0,0.63,1.97,26.73,X40,ATH,IT
51,MEDANTA,1486.00,6.42,-8.11,35.03,24.08,40864.0,-10297.0,116653.0,-15.60,46.0,X-SC,9.47,91.0,-0.25,0.87,6.84,XY24,NTT,HEALTHCARE


In [171]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,MEDANTA,1486.0,6.42,-8.11,35.03,24.08,40864.0,-10297.0,116653.0,-15.60,46.0,X-SC,9.47,91.0,-0.25,0.87,6.84,XY24,NTT,HEALTHCARE
68,STARHEALTH,761.0,5.26,-8.24,60.48,47.26,156017.0,-23158.0,257965.0,25.27,65.0,H-SC,3.31,174.0,-0.15,1.92,38.21,XY24,NTT,INSURANCE
15,CAMPUS,393.0,4.97,-11.87,51.24,33.28,75761.0,-19920.0,147855.0,-29.02,55.0,M-SC,5.77,221.0,-0.26,1.10,14.60,XY24,NTT,FOOTWEAR
70,SYMPHONY,1306.0,4.01,-29.87,42.55,-0.03,51069.0,-51119.0,120022.0,-43.61,60.0,M-SC,12.91,206.0,-1.00,0.89,10.68,OX40N,NTT,DURABLES
66,SIS,528.0,2.56,-23.10,57.68,21.26,49251.0,-25645.0,85387.0,2016.06,55.0,H-SC,4.81,163.0,-0.52,0.64,15.37,OX40N,NTT,MISC


In [172]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.00,-9.17,-5.49,31.42,24.20,90643.0,-16762.0,288488.0,34.71,38.0,X-SC,2.50,99.0,-0.18,2.15,20.82,X40N,NTT,FINANCE
50,MASFIN,398.61,-6.53,-8.14,32.87,22.05,29583.0,-7980.0,90000.0,-20.96,42.0,H-SC,3.38,168.0,-0.27,0.67,30.82,XR,ATH,FINANCE
46,JIOFIN,387.00,-5.99,-23.04,61.76,24.48,127074.0,-61610.0,205755.0,-24.33,17.0,X-LC,7.67,37.0,-0.48,1.53,19.26,XY24,BTT,FINANCE
0,5PAISA,593.00,-5.98,-37.43,80.44,12.90,109976.0,-81778.0,136718.0,101.92,41.0,H-SC,6.26,173.0,-0.74,1.02,13.68,OX40N,NTT,FINANCE
29,GREENPANEL,537.00,-5.85,-42.51,148.97,43.14,156801.0,-77821.0,105257.0,135.79,44.0,M-SC,17.38,231.0,-0.50,0.78,2.93,XY24,NTT,MISC


In [173]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.0,-0.9,-0.47,46.25,45.57,120435.0,-1219.0,260400.0,-36.01,66.0,H-MC,6.87,121.0,-0.01,1.94,15.63,AR,NTT,ELECTRICAL


In [174]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,-0.79,-10.68,132.78,107.91,175100.0,-15772.0,131872.0,-27.44,36.0,M-SC,10.86,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
72,TATAELXSI,9161.00,1.17,-22.38,70.09,32.03,71726.0,-29501.0,102334.0,-16.04,49.0,H-SC,6.52,157.0,-0.41,0.76,14.62,OX40N,NTT,IT
66,SIS,528.00,2.56,-23.10,57.68,21.26,49251.0,-25645.0,85387.0,2016.06,55.0,H-SC,4.81,163.0,-0.52,0.64,15.37,OX40N,NTT,MISC
48,KANSAINER,340.00,-0.34,-25.02,51.34,13.47,103806.0,-67473.0,202194.0,-69.09,40.0,H-SC,4.98,159.0,-0.65,1.50,2.81,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-2.09,-26.44,36.07,0.09,26740.0,-26644.0,74133.0,72.04,33.0,M-SC,6.50,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES


In [175]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [176]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.62,2.3,16.55,19.23,25474.0,3460.0,153920.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.15,24.06,XY25,ATH,MINING


In [177]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-0.90,-0.47,46.25,45.57,120435.0,-1219.0,260400.0,-36.01,66.0,H-MC,6.87,121.0,-0.01,1.94,15.63,AR,NTT,ELECTRICAL
84,WIPRO,420.00,1.35,-1.53,74.93,72.27,125030.0,-2585.0,166863.0,-15.61,44.0,M-LC,2.55,101.0,-0.02,1.24,7.25,XR,NTT,IT
39,INDIAMART,4810.62,0.46,-6.10,120.15,106.72,139153.0,-7520.0,115816.0,-53.49,48.0,H-SC,9.08,138.0,-0.05,0.86,15.86,AR,ATH,MISC
26,FINCABLES,1641.55,-0.79,-10.68,132.78,107.91,175100.0,-15772.0,131872.0,-27.44,36.0,M-SC,10.86,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-2.55,-8.47,48.18,35.63,92423.0,-17746.0,191828.0,-21.17,37.0,H-MC,5.51,119.0,-0.19,1.43,6.79,AR,ATH,PHARMA


In [178]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,387.00,-5.99,-23.04,61.76,24.48,127074.0,-61610.0,205755.0,-24.33,17.0,X-LC,7.67,37.0,-0.48,1.53,19.26,XY24,BTT,FINANCE
44,ITC,452.00,-5.04,-25.03,47.76,10.78,86052.0,-60143.0,180175.0,-55.46,17.0,X-LC,18.29,1.0,-0.70,1.34,0.00,X40,NTT,FMCG
13,BERGEPAINT,680.00,-0.12,-17.63,47.07,21.14,88140.0,-40087.0,187252.0,-23.43,19.0,X-MC,7.30,75.0,-0.45,1.39,0.00,XY24,NTT,PAINTS
12,BATAINDIA,2096.00,-0.32,-44.96,145.66,35.20,103151.0,-57854.0,70816.0,-7.15,20.0,X-SC,19.88,93.0,-0.56,0.53,0.64,X40,NTT,FOOTWEAR
6,ASIANPAINT,3460.25,-1.02,-15.99,43.96,20.95,92983.0,-40251.0,211517.0,-16.80,23.0,X-LC,10.58,36.0,-0.43,1.57,13.95,X40,ATH,PAINTS


In [179]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,0.80,-13.42,36.92,18.55,120912.0,-50741.0,327496.0,-25.57,49.0,X-LC,0.53,3.0,-0.42,2.44,11.42,X40,ATH,IT
81,VBL,671.64,-0.27,-5.91,44.19,35.67,131324.0,-18662.0,297180.0,-17.02,37.0,X-LC,1.82,5.0,-0.14,2.21,7.13,X40N,ATH,FMCG
34,HINDUNILVR,2922.00,-2.13,-9.66,25.82,13.67,59964.0,-24823.0,232240.0,-16.44,43.0,X-LC,2.24,9.0,-0.41,1.73,9.64,XY25,NTT,FMCG
5,ANGELONE,3033.00,-9.17,-5.49,31.42,24.20,90643.0,-16762.0,288488.0,34.71,38.0,X-SC,2.50,99.0,-0.18,2.15,20.82,X40N,NTT,FINANCE
36,ICICIGI,2252.93,-1.13,-4.39,24.82,19.34,47488.0,-8779.0,191330.0,-24.51,32.0,X-MC,2.77,71.0,-0.18,1.42,10.85,X40,ATH,INSURANCE


In [180]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-4.24,-32.45,143.00,64.15,229858.0,-77211.0,160740.0,-60.68,29.0,X-SC,7.29,84.0,-0.34,1.20,0.0,XY24,BTT,CEMENT
10,BAJAJHLDNG,14451.52,-1.94,-5.94,37.38,29.22,66844.0,-11302.0,178823.0,-8.67,36.0,X-LC,13.85,29.0,-0.17,1.33,0.0,X40,ATH,FINANCE
23,DIXON,18931.72,-2.73,-29.55,86.32,31.26,140336.0,-68192.0,162576.0,-67.12,26.0,X-MC,27.58,56.0,-0.49,1.21,0.0,X40N,ATH,IT
13,BERGEPAINT,680.00,-0.12,-17.63,47.07,21.14,88140.0,-40087.0,187252.0,-23.43,19.0,X-MC,7.30,75.0,-0.45,1.39,0.0,XY24,NTT,PAINTS
44,ITC,452.00,-5.04,-25.03,47.76,10.78,86052.0,-60143.0,180175.0,-55.46,17.0,X-LC,18.29,1.0,-0.70,1.34,0.0,X40,NTT,FMCG


In [181]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.55,-31.27,107.84,42.84,48179.0,-20330.0,44676.0,-54.82,47.0,X-SC,21.61,83.0,-0.42,0.33,5.01,XY24,NTT,MISC
12,BATAINDIA,2096.00,-0.32,-44.96,145.66,35.20,103151.0,-57854.0,70816.0,-7.15,20.0,X-SC,19.88,93.0,-0.56,0.53,0.64,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,-3.90,-51.31,212.81,52.31,150412.0,-74481.0,70679.0,-48.12,44.0,X-SC,11.45,92.0,-0.50,0.53,5.10,X40N,NTT,FOOTWEAR
51,MEDANTA,1486.00,6.42,-8.11,35.03,24.08,40864.0,-10297.0,116653.0,-15.60,46.0,X-SC,9.47,91.0,-0.25,0.87,6.84,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,1.04,-19.86,78.85,43.32,102915.0,-32352.0,130520.0,-31.14,46.0,X-SC,9.01,90.0,-0.31,0.97,5.98,X40N,ATH,ELECTRICAL


In [182]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,452.00,-5.04,-25.03,47.76,10.78,86052.0,-60143.0,180175.0,-55.46,17.0,X-LC,18.29,1.0,-0.70,1.34,0.00,X40,NTT,FMCG
75,TMPV,600.00,1.50,-27.18,68.87,22.98,113294.0,-61394.0,164504.0,-24.30,53.0,X-LC,4.55,2.0,-0.54,1.22,5.17,XY24,NTT,AUTO
73,TCS,4311.59,0.80,-13.42,36.92,18.55,120912.0,-50741.0,327496.0,-25.57,49.0,X-LC,0.53,3.0,-0.42,2.44,11.42,X40,ATH,IT
81,VBL,671.64,-0.27,-5.91,44.19,35.67,131324.0,-18662.0,297180.0,-17.02,37.0,X-LC,1.82,5.0,-0.14,2.21,7.13,X40N,ATH,FMCG
42,INFY,1972.00,-0.49,11.17,20.76,34.24,72209.0,34941.0,347829.0,-13.50,49.0,X-LC,7.13,6.0,0.48,2.59,20.37,X40,NTT,IT


In [183]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,-1.05,-34.37,102.47,32.89,51925.0,-26534.0,50673.0,-725.94,48.0,L-MC,11.26,259.0,-0.51,0.38,39.55,XR,NTT,BANKS
18,COALINDIA,484.83,-5.62,2.30,16.55,19.23,25474.0,3460.0,153920.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.15,24.06,XY25,ATH,MINING
68,STARHEALTH,761.00,5.26,-8.24,60.48,47.26,156017.0,-23158.0,257965.0,25.27,65.0,H-SC,3.31,174.0,-0.15,1.92,38.21,XY24,NTT,INSURANCE
33,HCLTECH,1908.19,-2.27,9.19,13.41,23.83,35423.0,22228.0,264152.0,14.03,53.0,X-LC,7.32,13.0,0.63,1.97,26.73,X40,ATH,IT
42,INFY,1972.00,-0.49,11.17,20.76,34.24,72209.0,34941.0,347829.0,-13.50,49.0,X-LC,7.13,6.0,0.48,2.59,20.37,X40,NTT,IT


In [184]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SATIN,274.00,0.28,-14.27,77.12,51.83,182656.0,-39438.0,236846.0,-23.78,62.0,H-SC,3.11,148.0,-0.22,1.76,15.38,XY24,NTT,FINANCE
27,GILLETTE,11206.78,-1.43,2.49,29.20,32.42,75984.0,6324.0,260220.0,-15.80,63.0,X-SC,12.67,87.0,0.08,1.94,17.97,X40,ATH,FMCG
1,ABB,7934.00,-0.90,-0.47,46.25,45.57,120435.0,-1219.0,260400.0,-36.01,66.0,H-MC,6.87,121.0,-0.01,1.94,15.63,AR,NTT,ELECTRICAL
33,HCLTECH,1908.19,-2.27,9.19,13.41,23.83,35423.0,22228.0,264152.0,14.03,53.0,X-LC,7.32,13.0,0.63,1.97,26.73,X40,ATH,IT
68,STARHEALTH,761.00,5.26,-8.24,60.48,47.26,156017.0,-23158.0,257965.0,25.27,65.0,H-SC,3.31,174.0,-0.15,1.92,38.21,XY24,NTT,INSURANCE


In [185]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.92
1,25,44.92
2,50,76.80


In [186]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,45.06
MC,30.63
LC,24.36


In [187]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,26.33
X40,24.13
X40N,14.84
AR,8.92
XR,8.72
XY25,8.66
OX40N,7.51
SR,0.94


In [188]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
X-MC,23.96
H-SC,23.08
X-LC,20.85
M-SC,11.04
X-SC,10.19
H-MC,4.83
M-MC,1.46
M-LC,1.24
L-LC,1.15


In [189]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-15.61,55.02
FINANCE,13.08,-18.85,65.65
IT,13.05,-22.07,83.05
MISC,6.91,-39.86,97.07
ELECTRICAL,6.00,-16.47,58.97
PAINTS,5.40,-27.00,45.87
INSURANCE,4.88,-3.31,38.67
PHARMA,4.06,-8.01,42.01
BANKS,2.87,-38.73,122.79


In [190]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3430540.0,21
AR,1373323.0,10
XR,1320767.0,12
X40,1267888.0,15
X40N,1126885.0,10
OX40N,783363.0,10
XY25,457261.0,6
SR,295279.0,2


In [191]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3692288.0,24
X-MC,1811503.0,16
M-SC,1575740.0,15
X-LC,1088035.0,12
X-SC,931115.0,9
H-MC,430803.0,3
L-SC,190899.0,2
M-LC,125030.0,1
H-LC,82822.0,1


In [192]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260203.0      6
           AR         947037.0      5
M-SC       XY24       898627.0      6
H-SC       XR         853758.0      7
X-MC       X40        614330.0      7
           XY24       494496.0      3
X-LC       X40        474423.0      6
X-MC       X40N       455777.0      4
X-SC       X40N       433079.0      4
M-SC       OX40N      352455.0      5
H-SC       OX40N      336011.0      4
X-SC       XY24       318901.0      3
H-SC       SR         295279.0      2
X-MC       XY25       246900.0      2
X-LC       XY24       240368.0      2
           X40N       238029.0      2
H-MC       XY24       217945.0      1
           AR         212858.0      2
M-SC       XR         194052.0      2
X-SC       X40        179135.0      2
X-LC       XY25       135215.0      2
M-SC       AR         130606.0      2
M-LC       XR         125030.0      1
L-SC       XR          96002.0      1
           OX40N       94897.0      1
H-LC       AR          82822.0      1
L-MC       XR          51925.0      1
M-MC       XY25        49672.0      1
L-LC       XY25        25474.0      1

In [193]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
